In [38]:


import glob
genSavePath = "F:/PythonProjects/NietoExcercise-1/SavedResults/"
day = 25 # Which month and days result you want to visualize in table
month = 10

curSavePath = f"{genSavePath}{day}-{month}-2/"
savedResults = glob.glob(curSavePath+"savedBestSeed*")

skipSubjects = []
onlySignificant = True
minAcc = 0.55


In [39]:
import pandas as pd
from array import ArrayType
from requests import delete
from tabulate import tabulate
from prettytable import PrettyTable as pt
import numpy as np


nameDict = dict()
nrOfNames = 0
kernelDict = dict()
nrOfKernel = 0
nrOfCombination = 0
currSeed = 0
currSubj = 0
combinationList = []
combinationResult = []


results = []
for path in savedResults:
    results.append(np.load(path, allow_pickle=True))

subjectAmount = 10
for x in range(subjectAmount):
    combinationResult.append([])

nrOfSeedsPerSubject = np.zeros(subjectAmount)
for s, subject in enumerate(results):
    currSubj = 0
    currSeed = 0
    subjectBestArray = np.empty([len(subject)], dtype=object)

    currSeed = subject[0]
    currSubj = subject[1]

    if currSubj in skipSubjects:
        continue
    nrOfSeedsPerSubject[currSubj] += 1

    for aResults in subject[2]:
        for aResults2 in aResults:
            # If the np array comes from svm results
            if len(aResults2) < 6:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]

                akernel = aResults2[2]
                aC = round(aResults2[3], 2)

                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [nameDict[aName], kernelDict[akernel], aC] not in combinationList:
                    combinationList.append([nameDict[aName], kernelDict[akernel], aC])
                    nrOfCombination += 1

                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aC]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])

                    # If the np array comes from NN results
            else:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                akernel = aResults2[2]
                aLz = round(aResults2[3], 2)
                areg = round(aResults2[4], 2)
                adrp = round(aResults2[5], 2)
                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [
                    nameDict[aName],
                    kernelDict[akernel],
                    aLz,
                    areg,
                    adrp,
                ] not in combinationList:
                    combinationList.append(
                        [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                    )
                    nrOfCombination += 1
                # combResult = []
                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])

                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])


# Making table

# headers = [f"{x[0]}:{x[1]}:{x[2]}:" for x in combinationList]
headers = np.arange(nrOfCombination)
subjectResultArray = np.empty([10, nrOfCombination], dtype=object)
subjectNrList = []

comboAverage = np.zeros([1, nrOfCombination])
comboTotal = np.zeros([1, nrOfCombination])
nrOfSubjects = np.zeros(nrOfCombination)

topTen = []
lowestTopRes = [0, 0]
for subNr, cResult in enumerate(combinationResult):
    resArray = np.zeros([nrOfCombination])
    if len(cResult) != 0:
        for res in cResult:
            if onlySignificant == True:
                if res[1] < minAcc:
                    continue
            comboAverage[0, res[0]] = comboAverage[0, res[0]] + res[1]
            comboTotal[0, res[0]] = comboTotal[0, res[0]] + res[1]
            nrOfSubjects[res[0]] += 1
            subjectResultArray[subNr, res[0]] = res[1]

            if len(topTen) < 10 or topTen[0][0] < res[1]:

                if len(topTen) == 10:
                    topTen.reverse()
                    topTen.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topTen.append([res[1], subNr, dataF, kernel, cVal])
                topTen.sort()


# Print the table!
comboAverage = comboAverage / nrOfSubjects
print(comboAverage.shape)
print(subjectResultArray.shape)
pd.set_option("display.max_columns", None)
table = pd.DataFrame(np.array(subjectResultArray), columns=headers)
display(table)


(1, 33)
(10, 33)


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_13516\1648509775.py:170: RuntimeWarning: invalid value encountered in divide
  comboAverage = comboAverage / nrOfSubjects


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,0.6,None,None,None,None,None,None,0.85,0.85,0.9,0.9,0.75,0.7,0.7,0.7,0.75,0.75,0.7,0.6,None,None,None,0.7,0.6,0.7,0.65,0.65,0.65,0.65,0.65,0.65,0.65,0.65
2,None,None,None,0.583333,None,None,None,None,None,None,0.583333,None,None,None,None,None,None,None,0.666667,0.583333,None,None,None,None,None,None,None,None,None,0.625,None,0.583333,0.625
3,0.611111,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.555556,None,None,None,None,None,None,None,None,None,None
4,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.625,0.666667,None,None,None,None,None,None,None,None,0.583333,0.583333,None,None,None,None,0.583333,0.583333
5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,None,None,None,0.590909,0.590909,0.636364,None,None,None,None,None,0.590909,None,0.590909,None,None,None,None,None,None,None,None,None,None,0.636364,0.590909,0.590909,None,None,None,None,None,None
7,0.583333,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.583333,None
8,0.6,None,0.6,0.6,0.65,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.55,None,None,0.55,None,None,None,None,None,None,None,None
9,None,None,None,None,None,0.583333,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [40]:
print("Top ten highest accuracies")
topTen.reverse()
for top in topTen:
    print(
        f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}")


Top ten highest accuracies
Accuracy 0.9 : subject 1, feature:dataWCV, kernelsigmoid, C:5.0
Accuracy 0.9 : subject 1, feature:dataWCV, kernelsigmoid, C:3.88
Accuracy 0.85 : subject 1, feature:dataWCV, kernelsigmoid, C:2.75
Accuracy 0.85 : subject 1, feature:dataWCV, kernelsigmoid, C:1.62
Accuracy 0.75 : subject 1, feature:dataHRCV, kernelrbf, C:5.0
Accuracy 0.75 : subject 1, feature:dataHRCV, kernelrbf, C:3.88
Accuracy 0.75 : subject 1, feature:dataHRCV, kernellinear, C:0.5
Accuracy 0.7 : subject 1, feature:dataHRCV, kernelrbf, C:2.75
Accuracy 0.7 : subject 1, feature:dataHRCV, kernelrbf, C:1.62
Accuracy 0.7 : subject 1, feature:dataHRCV, kernelrbf, C:0.5


In [41]:
subjAv = np.zeros(subjectResultArray.shape[0])
for s, subj in enumerate(subjectResultArray):
    subjAv[s] = 0
    if all(subj == None):
        continue
    subjAv[s] = np.mean(subj[np.where(subj != None)])
print(subjAv)
pd.set_option('display.max_columns', None)
table = pd.DataFrame(np.array(comboAverage), columns=headers)
display(table)


print(
    f"Max average accuracy {np.max(np.array(comboAverage[np.where(np.isnan(comboAverage)==False)]))}")
maxVal = np.max(
    np.array(comboAverage[np.where(np.isnan(comboAverage) == False)]))
maxInd = int(np.where(comboAverage == maxVal)[1][0])
print(maxInd)
print(list(nameDict.keys())[combinationList[maxInd][0]])
print(list(kernelDict.keys())[combinationList[maxInd][1]])
print(f"C: {combinationList[maxInd][2]}")


[0.         0.70625    0.60714286 0.58333333 0.60416667 0.
 0.60227273 0.58333333 0.59166667 0.58333333]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32
0,0.598611,NaN,0.6,0.591414,0.620455,0.609848,NaN,0.85,0.85,0.9,0.741667,0.670455,0.7,0.645455,0.7,0.6875,0.708333,0.7,0.633333,0.583333,NaN,0.55,0.627778,0.6,0.628788,0.608081,0.608081,0.65,0.65,0.6375,0.65,0.6,0.619444


Max average accuracy 0.9
9
dataWCV
sigmoid
C: 3.88


In [42]:
# table = pd.DataFrame(np.array(comboTotal), columns=headers)
# display(table)

# maxVal2 = np.max(np.array(comboTotal[np.where(np.isnan(comboTotal) == False)]))
# maxInd2 = int(np.where(comboTotal == maxVal2)[1])
# print(maxInd2)
# print(nrOfSubjects[maxInd2])
# print(
#     f"Max total significant accuracy then averaged {maxVal2/nrOfSubjects[maxInd2]} ")

# print(list(nameDict.keys())[combinationList[maxInd2][0]])
# print(list(kernelDict.keys())[combinationList[maxInd2][1]])
# print(f"C: {combinationList[maxInd2][2]} ")


In [43]:


# maxAcc = np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])])
# print(f"Max single accuracy position {np.where(subjectResultArray == maxAcc)}")
# print(np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])]))
